In [1]:
import heapq

class Agglomerative:
    def __init__(self, **kwargs):
        if 'n_clusters' in kwargs:
            self.n_clusters = int(kwargs['n_clusters'])
        self.labels_ = []
        if 'linkage' in kwargs:
            if kwargs['linkage'] == 'complete':
                self.helper = self.complete
            elif kwargs['linkage'] == 'single':
                self.helper = self.single
            elif kwargs['linkage'] == 'average':
                self.helper = self.average
            elif kwargs['linkage'] == 'average_group':
                self.helper = self.average_group
    
    def euclidian_distance(self, A, B):
        res = 0
        for i in range(len(A)):
            res += (A[i]-B[i]) ** 2
        return res ** 0.5
    
    def complete(self, data, a, b):
        result = None
        for i in a:
            for j in b:
                dist = self.euclidian_distance(data[i],data[j])
                if result is None :
                    result = dist
                elif dist > result :
                    result = dist
        return result
    
    def single(self, data, a, b):
        result = None
        for i in a:
            for j in b:
                dist = self.euclidian_distance(data[i],data[j])
                if result is None :
                    result = dist
                elif dist < result :
                    result = dist
        return result
    
    def average_group(self, data, a, b):
        result = 0
        for i in a:
            for j in b:
                dist = self.euclidian_distance(data[i],data[j])
                result += dist
        return result / (len(a) * len(b))
    
    def average(self, data, a, b):
        #find each centroid
        cen = [[0 for i in range(len(data[a[0]]))] for j in range(2)]
        for i in a:
            for j in range(len(data[i])):
                cen[0][j] += data[i][j]
        for j in range(len(data[a[0]])):
            cen[0][j] = cen[0][j] / len(a)
        for i in b:
            for j in range(len(data[i])):
                cen[1][j] += data[i][j]
        for j in range(len(data[b[0]])):
            cen[1][j] = cen[1][j] / len(b)
        return self.euclidian_distance(cen[0],cen[1])
    
    def fit(self, data):
        clusters = [[i] for i in range(len(data))]
        counter = 0
        prio = []
        cluster_mapping = dict()
        for i in range(len(clusters)) :
            cluster_mapping.update({i : [i]})
        
        for i in range(len(clusters)) :
            for j in range(i+1, len(clusters)):
                heapq.heappush(prio, (self.helper(data,cluster_mapping[i],cluster_mapping[j]), (i,j)))

        while(len(cluster_mapping) > self.n_clusters):
            counter += 1
            #keluarkan pertama
            first = heapq.heappop(prio)
            #aglo
            popped1 = cluster_mapping.pop(first[1][0])
            popped2 = cluster_mapping.pop(first[1][1])

            #delete unnecesarry data
            prio = list(filter(lambda x: not((x[1][0] == first[1][0]) or (x[1][1] == first[1][0]) or (x[1][0] == first[1][1]) or (x[1][1] == first[1][1])),prio)) 

            heapq.heapify(prio)

            newpopped = popped1 + popped2

            val = list(cluster_mapping.values())
            #add new data
            for key, value in cluster_mapping.items():
                heapq.heappush(prio, (self.helper(data,value,newpopped), (key,first[1][0])))

            cluster_mapping[first[1][0]] = newpopped
        
        #transposing
        self.labels_ = [0 for i in range(len(data))]
        cluster_mapping = list(cluster_mapping.values())
        for key in range(len(cluster_mapping)):
            for v in cluster_mapping[key]:
                self.labels_[v] = key
            
        

hehe = Agglomerative(n_clusters=3, linkage='average_group')
A = [[2, 10],[2, 5],[8, 4],[5, 8],[7, 5],[6,4],[1,2],[4,9]]
hehe.fit(A)
print(hehe.labels_)


[1, 2, 0, 1, 0, 0, 2, 1]


In [2]:
from sklearn import datasets
iris = datasets.load_iris()
hehe.fit(iris.data.tolist())
print(hehe.labels_)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2]


In [3]:
print (iris.target)

calc = [[0 for j in range(3)] for i in range(3)]

for i in range(len(hehe.labels_)):
    calc[iris.target[i]][hehe.labels_[i]] += 1

print(calc)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[50, 0, 0], [0, 0, 50], [0, 36, 14]]
